In [ ]:
!git clone --recursive https://github.com/rezazad68/fewshot-segmentation

Cloning into 'fewshot-segmentation'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 53 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [ ]:
!pip uninstall -yq tf-nightly-gpu  tensorflow keras tensorflow-gpu keras-preprocessing keras-applications 
!pip install tensorflow-gpu==1.13.1
!pip install numpy keras==2.3 keras-applications==1.0.6 keras-preprocessing==1.0.5 blend_modes youtube_dl pafy

  Using cached https://files.pythonhosted.org/packages/7b/b1/0ad4ae02e17ddd62109cd54c291e311c4b5fd09b4d0678d3d6ce4159b0f0/tensorflow_gpu-1.13.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/79/4c/7c3275a01e12ef9368a892926ab932b33bb13d55794881e3573482b378a7/Keras_Preprocessing-1.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1b/18/2e1ef121e5560ac24c7ac9e363aa5fa7006c40563c989e7211aba95b793a/Keras-2.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/c4/2ff40221029f7098d58f8d7fb99b97e8100f3293f9856f0fb5834bef100b/Keras_Applications-1.0.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fc/94/74e0fa783d3fc07e41715973435dd051ca89c550881b3454233c39c73e69/Keras_Preprocessing-1.0.5-py2.py3-none-any.wh

In [ ]:
%cd fewshot-segmentation/source_code/

/content/fewshot-segmentation/source_code


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import model as  M
import matplotlib.pyplot as plt
import utilz as U
import numpy as np
from parser_utils import get_parser
import pickle
import random 
import cv2
import copy
from PIL import Image
import numpy
from blend_modes import *
import pafy

## Get AI options
options =type('Expando', (object,), {})()
options.img_h = 224
options.img_w = 224
options.kshot = 1
options.nway = 5
options.learning_rate = 0.0001

#we are ready for 5 class 5 shot queries.
#each folder in folder_with_sample_folders must have 
#up to 5 jpg color images 
#with a black and white mask png file per image
#query must be a colored image
folder_with_sample_folders = "/content/drive/MyDrive/fss/w5s1-as_w1s5/test/"
video_url = 'https://youtu.be/KZSd6TyuvsQ'

target_in_image_path = "/content/drive/MyDrive/fss/w5s1-as_w1s5/query/" + "img" + '.jpg'
target_out_mask_path_base = "/content/drive/MyDrive/fss/w5s1-as_w1s5/query/" + "msk" 
model_weights_path = '/content/drive/MyDrive/fss/w5s1-as_w1s5/fewshot_DOGLSTM-w5-s1.h5'


def data(opt, folder_with_sample_folders, target_in_image_path):
    support = np.zeros([opt.nway, opt.kshot, opt.img_h, opt.img_w, 3], dtype = np.float32)
    smasks  = np.zeros([opt.nway, opt.kshot, 56,        56,        1], dtype = np.float32)

    setX = os.listdir(folder_with_sample_folders)
    print(opt.nway)
    for idx in range(opt.nway):
        print(idx)
        for idy in range(0, opt.kshot): # For support set 
            print(idy)
            s_img = cv2.imread(folder_with_sample_folders + setX[idx] + '/' + str(idy+1) + '.jpg' )
            print(folder_with_sample_folders + setX[idx] + '/' + str(idy+1) + '.jpg')
            s_msk = cv2.imread(folder_with_sample_folders + setX[idx] + '/' + str(idy+1) + '.png' )
            print(folder_with_sample_folders + setX[idx] + '/' + str(idy+1) + '.png' )
            s_img = cv2.resize(s_img,(opt.img_h, opt.img_w))
            s_msk = s_msk /255
            s_msk = cv2.resize(s_msk,(56,        56))        
            s_msk = np.where(s_msk > 0.5, 1., 0.)
            support[idx, idy] = s_img
            smasks[idx, idy]  = s_msk[:, :, 0:1] 

    support = support /255.
    return support, smasks   

def read_video(opt):
    print("model loading....")
    model = M.my_model(encoder = 'VGG_b345', input_size = (options.img_h, options.img_w, 3), k_shot = options.kshot, learning_rate = options.learning_rate)
    model.summary()
    model.load_weights(model_weights_path)
    overall_miou = 0.0
    
    ## Get an episode for test 
    support, smask = data(opt, folder_with_sample_folders, target_in_image_path)
    print("model initiated")
    urlPafy  = pafy.new(video_url)
    videoplay   = urlPafy.getbest()

    # open url with opencv
    cap = cv2.VideoCapture(videoplay.url)
    if not cap.isOpened():
        print('video not opened')
        exit(-1)

    fidx =0
    while os.path.isfile(target_out_mask_path_base + str(fidx)+ ".avi"):
      fidx = fidx + 1
    out_name  = target_out_mask_path_base + str(fidx) + ".avi"
    colorwriter = cv2.VideoWriter(out_name, cv2.VideoWriter_fourcc(*'XVID'), 15, (224, 224), 1)
    frid = 0
    while True:
        frid = frid + 1
        # read frame
        ret, frame = cap.read()
        # check if frame is empty
        if not ret:
            break
        print("frame " + str(frid) )
        frame = cv2.resize(frame,(opt.img_h, opt.img_w))
        query   = np.zeros([opt.nway,            opt.img_h, opt.img_w, 3], dtype = np.float32)      
                    
        for idx in range(opt.nway):
          query[idx] = frame.copy()
      
        query   = query   /255.
        Es_mask = model.predict([support, smask, query])
        # Compute MIOU for episode
        Es_mask = np.where(Es_mask > 0.6, 1 , 0.)
        Es_mask = Es_mask * 255
        O = Es_mask[0]
        for idx in range(Es_mask.shape[0]):
          O = O + Es_mask[idx]
        
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2RGBA).astype(float)

        # Import foreground image
        O = cv2.resize(O, (options.img_h, options.img_w)).astype(numpy.uint8)

        O = cv2.cvtColor(O, cv2.COLOR_GRAY2RGBA)
        r = O.copy()
        r[:, :, 0] = 0
        r[:, :, 2] = 0

        O = r.astype(float)

        # Blend images
        opacity = 0.7  # The opacity of the foreground that is blended onto the background is 70 %.
        blended = soft_light(frame, O, opacity)

        # Display blended image
        blended = blended.astype(numpy.uint8)  # Convert image to OpenCV native display format
        blended = cv2.cvtColor(blended, cv2.COLOR_RGBA2BGR)

        colorwriter.write(blended)
    
    # release VideoCapture
    cap.release()
    colorwriter.release()
    print("done")

read_video(options) 

model loading....
Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_94 (InputLayer)           (None, 1, 224, 224,  0                                            
__________________________________________________________________________________________________
sequential_24 (Sequential)      (None, 1, 56, 56, 12 14714688    input_94[0][0]                   
__________________________________________________________________________________________________
input_95 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
time_distributed_416 (TimeDistr (None, 1, 56, 56, 12 1474688     sequential_24[1][0]              
_________________________________________________________________________

True

In [ ]:
!pip install pafy